# EDA Related to creating the Similarity Matrices


In [145]:
import os
import pandas as pd
import numpy as np
import re

In [146]:
data_dir = "../data/patientData"

In [26]:
# let's check some of the missing data
files = os.listdir(data_dir)
lab_files = [file for file in files if re.search("lab tests", file)]
lab_files

['lab tests_cross sect_antibodies.csv',
 'lab tests_cross sect_blood.csv',
 'lab tests_cross sect_breakdown products.csv',
 'lab tests_cross sect_enzymes.csv',
 'lab tests_cross sect_hepatitis.csv',
 'lab tests_cross sect_other lab.csv',
 'lab tests_cross sect_proteins.csv',
 'lab tests_time series.csv']

In [67]:
blood = pd.read_csv(data_dir + "/lab tests_cross sect_blood.csv")
print(len(blood.columns))
blood.head()

24


,subject_id,min_fibrinogen,max_fibrinogen,avg_fibrinogen,min_platelet,max_platelet,avg_platelet,min_inr,max_inr,avg_inr,...,max_ptt,avg_ptt,first_ferritin,first_iron,first_iron_bind,first_transferrin,last_ferritin,last_iron,last_iron_bind,last_transferrin
0,4,NaN,NaN,NaN,201.0,388.0,286.428571,1.0,1.1,1.050000,...,33.2,32.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52,NaN,NaN,NaN,47.0,115.0,72.583333,1.7,2.7,2.241667,...,48.4,42.633333,640.0,90.0,185.0,142.0,NaN,NaN,NaN,NaN
2,78,NaN,NaN,NaN,45.0,53.0,47.750000,1.2,1.2,1.200000,...,39.0,38.350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,117,79.0,253.0,158.25,24.0,98.0,52.565217,1.5,3.9,2.143750,...,150.0,56.434091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,140,NaN,NaN,NaN,65.0,86.0,72.666667,1.0,1.3,1.150000,...,31.5,30.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# let's check number of patients missing >50% of data
blood['total_count'] = blood.isnull().sum(axis=1)
round(blood.total_count.value_counts()/len(blood),4)*100

8     31.39
11    28.51
0     16.91
3     10.24
4      3.72
2      1.94
7      1.67
5      1.63
20     1.60
6      0.90
1      0.56
9      0.45
14     0.24
12     0.07
10     0.07
23     0.07
17     0.03
Name: total_count, dtype: float64

In [27]:
for file in lab_files:
    print(file)
    df = pd.read_csv(data_dir + "/" + file)
    
    print(len(df.columns))
    
    df['total_count'] = df.isnull().sum(axis=1)
    print(round(df.total_count.value_counts()/len(df),4)*100)
    
    print("\n ~~~~~~~~~~~~~~~~~~~~~~~~~~ \n")

lab tests_cross sect_antibodies.csv
5
0    100.0
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_blood.csv
24
8     31.39
11    28.51
0     16.91
3     10.24
4      3.72
2      1.94
7      1.67
5      1.63
20     1.60
6      0.90
1      0.56
9      0.45
14     0.24
12     0.07
10     0.07
23     0.07
17     0.03
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_breakdown products.csv
17
5     19.97
4     18.12
2     13.44
3     12.29
7     11.91
6     10.87
1      3.58
13     2.53
10     2.33
8      1.49
0      1.39
11     1.08
9      0.62
12     0.38
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_enzymes.csv
19
3     57.40
6     18.02
0     15.14
9      4.38
15     2.92
18     1.81
12     0.35
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab tests_cross sect_hepatitis.csv
5
0    100.0
Name: total_count, dtype: float64

 ~~~~~~~~~~~~~~~~~~~~~~~~~~ 

lab t

Only one that is kind of bad is proteins. Otherwise, I can just remove patients from the study who have less than 50% of variables. 

In [119]:
# lets test to see if we need to normalize our data for sklearn cosine similarity
from sklearn import preprocessing
blood_scaled = preprocessing.scale(blood.drop(columns = "subject_id"))
blood_scaled = pd.concat([blood.subject_id, pd.DataFrame(blood_scaled, columns = blood.columns[1:])], axis =1)
blood_scaled.head()

,subject_id,min_fibrinogen,max_fibrinogen,avg_fibrinogen,min_platelet,max_platelet,avg_platelet,min_inr,max_inr,avg_inr,...,max_ptt,avg_ptt,first_ferritin,first_iron,first_iron_bind,first_transferrin,last_ferritin,last_iron,last_iron_bind,last_transferrin
0,4,NaN,NaN,NaN,1.205404,0.662254,1.123674,-0.714467,-0.383549,-0.682573,...,-0.841565,-0.610264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52,NaN,NaN,NaN,-0.664753,-0.815391,-0.842730,1.087517,-0.044092,0.584222,...,-0.498187,0.133654,-0.059937,0.336823,-0.231159,-0.235295,NaN,NaN,NaN,NaN
2,78,NaN,NaN,NaN,-0.689040,-1.150974,-1.071083,-0.199615,-0.362332,-0.523116,...,-0.710540,-0.173227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,117,-0.876438,-0.416808,-0.71165,-0.944062,-0.907406,-1.026805,0.572664,0.210501,0.480133,...,1.797025,1.122416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,140,NaN,NaN,NaN,-0.446163,-0.972357,-0.841963,-0.714467,-0.341116,-0.576269,...,-0.879970,-0.757137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
# alright I am going to have to write the code myself 
# generate all combinations
import itertools 
from sklearn.metrics.pairwise import cosine_similarity

subjects = blood.subject_id[:5]
pairwise = list(itertools.combinations(subjects, 2))
pair1 = pairwise[0]

# extract patient rows and identify non-missing columns
row1 = blood_scaled.drop(columns = "subject_id").iloc[2]
columns1 = row1[~row1.isnull()].index
row2 = blood_scaled.drop(columns = "subject_id").iloc[3]
columns2 = row2[~row2.isnull()].index

# find intersection and compare
shared_cols = list(set(columns1) & set(columns2))

cosine_similarity([np.array(row1[shared_cols])], [np.array(row2[shared_cols])])[0][0]

0.19251735060129557

In [134]:
pd.concat([row1[shared_cols], row2[shared_cols]], axis = 1)

,2,3
min_platelet,-0.689040,-0.944062
avg_platelet,-1.071083,-1.026805
max_inr,-0.362332,0.210501
min_inr,-0.199615,0.572664
max_pt,-0.296417,-0.021170
min_ptt,0.930094,0.584657
min_pt,-0.264982,0.177497
avg_pt,-0.670983,0.147884
max_ptt,-0.710540,1.797025
avg_inr,-0.523116,0.480133


In [143]:
blood

,subject_id,min_fibrinogen,max_fibrinogen,avg_fibrinogen,min_platelet,max_platelet,avg_platelet,min_inr,max_inr,avg_inr,...,max_ptt,avg_ptt,first_ferritin,first_iron,first_iron_bind,first_transferrin,last_ferritin,last_iron,last_iron_bind,last_transferrin
0,4,NaN,NaN,NaN,201.0,388.0,286.428571,1.0,1.1,1.050000,...,33.2,32.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52,NaN,NaN,NaN,47.0,115.0,72.583333,1.7,2.7,2.241667,...,48.4,42.633333,640.0,90.0,185.0,142.0,NaN,NaN,NaN,NaN
2,78,NaN,NaN,NaN,45.0,53.0,47.750000,1.2,1.2,1.200000,...,39.0,38.350000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,117,79.0,253.0,158.250000,24.0,98.0,52.565217,1.5,3.9,2.143750,...,150.0,56.434091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,140,NaN,NaN,NaN,65.0,86.0,72.666667,1.0,1.3,1.150000,...,31.5,30.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,99791,NaN,NaN,NaN,243.0,361.0,301.777778,1.1,1.3,1.216667,...,34.2,33.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2876,99830,114.0,409.0,178.666667,64.0,649.0,266.243902,0.8,2.1,1.376190,...,80.9,43.014286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2877,99836,313.0,670.0,491.500000,101.0,323.0,191.684932,1.3,2.1,1.521429,...,45.3,34.617500,154.0,43.0,207.0,159.0,302.0,46.0,144.0,111.0
2878,99862,NaN,NaN,NaN,366.0,490.0,438.500000,1.3,1.3,1.300000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## EDA for Charts Events

In [149]:
# for testing
df = pd.read_csv(f"{data_dir}/chart events_categorical_activity.csv")
df.head()     

,subject_id,label,value,num_occurences,total_counts,frequency
0,4,Activity,Ambulate,1,10,0.1
1,4,Activity,Bedrest,5,10,0.5
2,4,Activity,Commode,4,10,0.4
3,4,Activity Tolerance,Good,3,10,0.3
4,4,Activity Tolerance,Tolerated Well,7,10,0.7


In [151]:
import prince
# generate the multi index category matrix
# pivot with two columns
nominal_mat = df.pivot_table(
                values='frequency', 
                index='subject_id', 
                columns=['label', 'value'])

# now I can actually replace the NaN's with 0 because this means 0 frequency
nominal_mat = nominal_mat.fillna(0)
display(nominal_mat.head())

# now - lets apply MCA
mca = prince.MCA(n_components=4)
mca = mca.fit(nominal_mat)
mca_mat = mca.transform(nominal_mat)
mca_mat.head()

label       Activity                                                       \
value       Ambulate Bed as Chair   Bedrest     Chair   Commode    Dangle   
subject_id                                                                  
4           0.100000          0.0  0.500000  0.000000  0.400000  0.000000   
52          0.000000          0.0  0.666667  0.000000  0.333333  0.000000   
78          0.166667          0.0  0.666667  0.000000  0.000000  0.166667   
117         0.000000          0.0  1.000000  0.000000  0.000000  0.000000   
140         0.066667          0.0  0.666667  0.133333  0.133333  0.000000   

label      Activity Tolerance                                     \
value                    Fair      Good      Poor Tolerated Well   
subject_id                                                         
4                    0.000000  0.300000  0.000000       0.700000   
52                   0.000000  0.000000  0.000000       1.000000   
78                   0.000000  0.200000  0.000000       0.800000   
117                  0.235294  0.088235  0.039216       0.637255   
140                  0.000000  0.166667  0.000000       0.833333   

label      Braden Activity                                                \
value              Bedfast Chairfast Walks Frequently Walks Occasionally   
subject_id                                                                 
4                 0.166667  0.166667         0.000000           0.666667   
52                0.750000  0.000000         0.000000           0.250000   
78                0.500000  0.000000         0.000000           0.500000   
117               1.000000  0.000000         0.000000           0.000000   
140               0.000000  0.555556         0.444444           0.000000   

label          Braden Mobility                                                \
value      Completely Immobile No Limitations Sl. Limited Slight Limitations   
subject_id                                                                     
4                     0.000000       0.500000    0.500000                0.0   
52                    0.000000       0.125000    0.875000                0.0   
78                    0.000000       0.750000    0.250000                0.0   
117                   0.111111       0.018519    0.148148                0.0   
140                   0.000000       0.555556    0.444444                0.0   

label                    
value      Very Limited  
subject_id               
4              0.000000  
52             0.000000  
78             0.000000  
117            0.722222  
140            0.000000

C:\Users\ABell\Anaconda3\lib\site-packages\prince\one_hot.py:35: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  default_fill_value=0
C:\Users\ABell\Anaconda3\lib\site-packages\pandas\core\sparse\frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
C:\Users\ABell\Anaconda3\lib\site-packages\pandas\core\frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-doc

,0,1,2,3
4,-0.011320,-0.012385,-0.013471,-0.010737
52,-0.012260,-0.012161,-0.014152,-0.010690
78,-0.012088,-0.012505,-0.016186,-0.011976
117,-0.010866,-0.017157,-0.013275,-0.014463
140,-0.011064,-0.011703,-0.013375,-0.013350


In [152]:
nominal_mat = df.pivot_table(
                values='frequency', 
                index='subject_id', 
                columns=['label', 'value'])

# now I can actually replace the NaN's with 0 because this means 0 frequency
nominal_mat = nominal_mat.fillna(0)
display(nominal_mat.head())

# now - lets apply MCA
pca = prince.PCA(n_components=4)
pca = pca.fit(nominal_mat)
pca_mat = pca.transform(nominal_mat)
pca_mat.head()

label       Activity                                                       \
value       Ambulate Bed as Chair   Bedrest     Chair   Commode    Dangle   
subject_id                                                                  
4           0.100000          0.0  0.500000  0.000000  0.400000  0.000000   
52          0.000000          0.0  0.666667  0.000000  0.333333  0.000000   
78          0.166667          0.0  0.666667  0.000000  0.000000  0.166667   
117         0.000000          0.0  1.000000  0.000000  0.000000  0.000000   
140         0.066667          0.0  0.666667  0.133333  0.133333  0.000000   

label      Activity Tolerance                                     \
value                    Fair      Good      Poor Tolerated Well   
subject_id                                                         
4                    0.000000  0.300000  0.000000       0.700000   
52                   0.000000  0.000000  0.000000       1.000000   
78                   0.000000  0.200000  0.000000       0.800000   
117                  0.235294  0.088235  0.039216       0.637255   
140                  0.000000  0.166667  0.000000       0.833333   

label      Braden Activity                                                \
value              Bedfast Chairfast Walks Frequently Walks Occasionally   
subject_id                                                                 
4                 0.166667  0.166667         0.000000           0.666667   
52                0.750000  0.000000         0.000000           0.250000   
78                0.500000  0.000000         0.000000           0.500000   
117               1.000000  0.000000         0.000000           0.000000   
140               0.000000  0.555556         0.444444           0.000000   

label          Braden Mobility                                                \
value      Completely Immobile No Limitations Sl. Limited Slight Limitations   
subject_id                                                                     
4                     0.000000       0.500000    0.500000                0.0   
52                    0.000000       0.125000    0.875000                0.0   
78                    0.000000       0.750000    0.250000                0.0   
117                   0.111111       0.018519    0.148148                0.0   
140                   0.000000       0.555556    0.444444                0.0   

label                    
value      Very Limited  
subject_id               
4              0.000000  
52             0.000000  
78             0.000000  
117            0.722222  
140            0.000000

,0,1,2,3
subject_id,,,,
4,5.873572,0.970705,2.639591,1.070375
52,2.649918,-1.808166,0.987571,1.101641
78,4.173878,-0.144994,3.182611,-0.547616
117,-2.241963,-0.238580,0.801331,0.499243
140,5.509665,-0.000466,1.755036,0.535418


In [153]:
# Let's compare the pairwise similarity of all of them
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(nominal_mat)[1]

array([0.80462467, 1.        , 0.78724303, ..., 0.69729171, 0.75420809,
       0.69966742])

In [154]:
cosine_similarity(mca_mat)[1]

array([0.99924382, 1.        , 0.99809736, ..., 0.18803165, 0.99111146,
       0.98878271])

In [155]:
cosine_similarity(pca_mat)[1]

array([ 0.75468357,  1.        ,  0.74326778, ..., -0.86069807,
        0.33963556, -0.20967568])